# Carga Data TERMINO X MATERIA

Creación del DataFrame asociado a los terminos por materia

Rev: 29-10-2020

In [1]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from tqdm import tqdm

from src.data import cleandata

In [2]:
path_raw = "../data/raw/pjud" 
archivos = os.listdir(path_raw) 
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
#creacion dataframe con los datos de Términos por Materia Penal de los años 2015 a 2020

dataframes = []

for archivo in archivos:
    if archivo.find("Términos por Materia Penal") != -1:
        df = pd.read_csv(f"{path_raw}/{archivo}", sep = ";", encoding = 'cp850', low_memory = True)
        dataframes.append(df) 
        
df_termino_materia = pd.concat(dataframes, axis = 0)

In [4]:
df_termino_materia.columns

Index(['N°', 'CÓD. CORTE', 'CORTE', 'CÓD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'TIPO CAUSA', 'CÓD. MATERIA', 'MATERIA', 'FECHA INGRESO',
       'FECHA TÉRMINO', 'MES TÉRMINO', 'AÑO TÉRMINO', 'MOTIVO DE TÉRMINO',
       'DURACIÓN CAUSA', 'TOTAL TÉRMINOS', 'SISTEMA'],
      dtype='object')

In [11]:
df_termino_materia['SISTEMA'].unique()

array(['SIAGJ', 'METGE'], dtype=object)

Elimino los registros asociados a Sistema METGE (METAS DE GESTION) ya que son causas para realizar cumplimiento de KPI

In [12]:
df_metge = df_termino_materia[df_termino_materia['SISTEMA']=='METGE']
df_termino_materia.drop(df_metge.index, axis=0, inplace=True)

In [13]:
# Estandarización de nombres de variables

df_termino_materia.rename(columns = {'CÓD. CORTE':'COD. CORTE',
                                     'CÓD. TRIBUNAL':'COD. TRIBUNAL',
                                     'CÓD. MATERIA':'COD. MATERIA',
                                     'MOTIVO DE TÉRMINO':'MOTIVO TERMINO',
                                     'DURACIÓN CAUSA':'DURACION CAUSA',
                                     'FECHA TÉRMINO':'FECHA TERMINO',
                                     'MES TÉRMINO':'MES TERMINO',
                                     'AÑO TÉRMINO':'AÑO TERMINO',
                                     'TOTAL TÉRMINOS':'TOTAL TERMINOS'
                                    },inplace = True)

df_termino_materia.drop(['N°','SISTEMA'], axis = 'columns', inplace = True)

In [14]:
# TRANSFORMAMOS DE FLOAT A INTEGER

df_termino_materia['COD. CORTE'] = df_termino_materia['COD. CORTE'].fillna(0).astype(np.int16)
df_termino_materia['COD. TRIBUNAL'] = df_termino_materia['COD. TRIBUNAL'].fillna(0).astype(np.int16)
df_termino_materia['COD. MATERIA'] = df_termino_materia['COD. MATERIA'].fillna(0).astype(np.int16)
df_termino_materia['DURACION CAUSA'] = df_termino_materia['DURACION CAUSA'].fillna(0).astype(np.int16)
df_termino_materia['AÑO TERMINO'] = df_termino_materia['AÑO TERMINO'].fillna(0).astype(np.int16)
df_termino_materia['TOTAL TERMINOS'] = df_termino_materia['TOTAL TERMINOS'].fillna(0).astype(np.int8)

In [15]:
# Transformamos formato fecha

df_termino_materia['FECHA INGRESO'] = df_termino_materia['FECHA INGRESO'].progress_apply(cleandata.convierte_fecha)
df_termino_materia['FECHA TERMINO'] = df_termino_materia['FECHA TERMINO'].progress_apply(cleandata.convierte_fecha)

100%|██████████| 3605987/3605987 [00:06<00:00, 549472.10it/s]


In [16]:
# Elimino espacios en las columnas tipo objetos

df_termino_materia = df_termino_materia.progress_apply(cleandata.elimina_espacios, axis=0)

100%|██████████| 15/15 [00:32<00:00,  2.18s/it]


In [17]:
# Elimino tildes de object

cols = df_termino_materia.select_dtypes(include = ["object"]).columns
df_termino_materia[cols] = df_termino_materia[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 7/7 [01:02<00:00,  8.94s/it]


In [18]:
# Categorizar variables

df_termino_materia['CORTE'] = df_termino_materia['CORTE'].astype('category')
df_termino_materia['MOTIVO TERMINO'] = df_termino_materia['MOTIVO TERMINO'].astype('category')

Dejo solo causas Ordinarias

In [22]:
df_termino_materia['TIPO CAUSA'].unique()

[Ordinaria, Militar, Extradición]
Categories (3, object): [Ordinaria, Militar, Extradición]

In [23]:
tipo_causa = df_termino_materia[df_termino_materia['TIPO CAUSA']!='Ordinaria']

In [24]:
df_termino_materia.drop(tipo_causa.index, axis=0, inplace=True)

In [25]:
# Reset el index para realizar feather

df_termino_materia.reset_index(inplace = True)

In [26]:
df_termino_materia

,level_0,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
0,0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018-06-01,jun-18,2018,SENTENCIA,60,1
1,1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018-06-01,jun-18,2018,SENTENCIA,60,1
2,2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018-08-29,ago-18,2018,SENTENCIA,149,1
3,3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018-03-14,mar-18,2018,SENTENCIA,63,1
4,4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018-05-29,may-18,2018,SENTENCIA,57,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3605458,3605982,721902,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,98-2015,Ordinaria,901,"CUASIDELITO DE LESIONES: ART 490, 491 INC 2° Y...",2015-12-17,2016-05-21,may-16,2016,SENTENCIA,156,1
3605459,3605983,721903,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,98-2015,Ordinaria,12077,NO DAR CUENTA DE ACCIDENTE DE TRANSITOART. 195...,2015-12-17,2016-05-21,may-16,2016,SENTENCIA,156,1
3605460,3605984,721904,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,98-2016,Ordinaria,10008,PORTE DE ARMA PROHIBIDA (ART. 14 INC. 1°),2016-07-27,2016-11-08,nov-16,2016,SENTENCIA,104,1
3605461,3605985,721905,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,99-2015,Ordinaria,12082,AMENAZASA CARABINEROS (ART. 417 COD. J.MILITAR).,2015-12-17,2016-01-18,ene-16,2016,SENTENCIA,32,1


In [27]:
# Guardamos dataset como archivo feather
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

df_termino_materia.to_feather(f'{path_interim}/TerminoMateria.feather')